## Still under construction :-)

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
def billboard_top_100(url):
    """
    Input the url and return a data frame with Top 100 songs from Billboard
    
    """
    billboard = requests.get(url)

    soup = BeautifulSoup(billboard.content, 'html.parser')

    # song titles
    songs = soup.find_all("span", class_ ="chart-element__information__song text--truncate color--primary")

    # artists
    artists = soup.find_all("span", class_ ="chart-element__information__artist text--truncate color--secondary")

    # last week
    last_week_ranks = soup.find_all("span", class_= "chart-element__meta text--center color--secondary text--last")

    # peak rank
    peak_ranks = soup.find_all("span", class_= "chart-element__meta text--center color--secondary text--peak")

    # weeks on chart
    weeks_on_chart = soup.find_all("span", class_= "chart-element__meta text--center color--secondary text--week")
    
    #

    for i in [songs, artists, last_week_ranks, peak_ranks, weeks_on_chart]:
        for j in range(len(i)):
            i[j] = i[j].getText()
        
    billboard = pd.DataFrame(
    {"songs": songs,
    "artists": artists,
    "last_week_rank": last_week_ranks,
    "peak_rank" : peak_ranks,
    "weeks_on_chart": weeks_on_chart}
    )

    return billboard


In [12]:
billboard = billboard_top_100("https://www.billboard.com/charts/hot-100")
billboard

,songs,artists,last_week_rank,peak_rank,weeks_on_chart
0,Stay,The Kid LAROI & Justin Bieber,2,1,13
1,Industry Baby,Lil Nas X & Jack Harlow,3,2,11
2,Fancy Like,Walker Hayes,5,3,16
3,Way 2 Sexy,Drake Featuring Future & Young Thug,4,1,5
4,Bad Habits,Ed Sheeran,6,2,15
...,...,...,...,...,...
95,Esta Danada,Ivan Cornejo,-,96,1
96,Whiskey And Rain,Michael Ray,-,97,1
97,TSU,Drake,84,9,5
98,Love All,Drake Featuring JAY-Z,87,10,5


In [6]:
def billboard_with_features(client_id, client_secret, billboard_url):
    
    """
    Explanations: In this function you will be able to extract a dataframe that consists 
    the Top Hot songs in the billboard. In addtion, by using the Spotify API you will be able to extract
    the audio features as well.
    
    client_id = spotify client ID
    client_secret = spotify secret key
    billboard_url = url from billboard 100
    """
    
    # initialize SpotiPy with user credentias
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))
    
    billboard = billboard_top_100(billboard_url)
    billboard_track_names = billboard['songs']
    
    # fetch the track names of the billboard_top_100
    billboard_tracks = billboard['songs'].tolist()
    
    # fetch the tracks_id from spotify API based on the tracks and make a new df
    billboard_tracks_id = []
    for track in billboard_tracks:
        results = sp.search(q='track: {0}'.format(track), limit=1)
        billboard_tracks_id.append(results['tracks']['items'][0]['id'])
    billboard_tracks_id = pd.DataFrame(billboard_tracks_id)
    billboard_tracks_id.columns = ['track_id']
    
    # Now fetch the audio features from spotify API
    spotify_audio_features = sp.audio_features(tracks = billboard_tracks_id['track_id'])
    spotify_audio_features_df = pd.DataFrame(spotify_audio_features)
    
    # Concat the df
    billboard_100 = pd.concat([billboard_tracks_id, spotify_audio_features_df, billboard_track_names], axis = 1)
    
    return billboard_100
    

    

In [7]:
billboard_100 = billboard_with_features("47749bd8cec54f04955b9db8069fc8c6", "621e6603f06d4acfb95010a79f92fc79", "https://www.billboard.com/charts/hot-100")
billboard_100

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,songs
0,5HCyWlXZPP0y6Gqq8TgA20,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,audio_features,5HCyWlXZPP0y6Gqq8TgA20,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,https://api.spotify.com/v1/tracks/5HCyWlXZPP0y...,https://api.spotify.com/v1/audio-analysis/5HCy...,141806,4,Stay
1,27NovPIUIRrOZoCHxABJwK,0.736,0.704,3,-7.409,0,0.0615,0.020300,0.000000,0.0501,0.894,149.995,audio_features,27NovPIUIRrOZoCHxABJwK,spotify:track:27NovPIUIRrOZoCHxABJwK,https://api.spotify.com/v1/tracks/27NovPIUIRrO...,https://api.spotify.com/v1/audio-analysis/27No...,212000,4,Industry Baby
2,3ZozFqJJlDBNIm4xqxn2ZD,0.649,0.760,1,-6.487,1,0.0574,0.114000,0.000000,0.3170,0.843,80.018,audio_features,3ZozFqJJlDBNIm4xqxn2ZD,spotify:track:3ZozFqJJlDBNIm4xqxn2ZD,https://api.spotify.com/v1/tracks/3ZozFqJJlDBN...,https://api.spotify.com/v1/audio-analysis/3Zoz...,161853,4,Fancy Like
3,0k1WUmIRnG3xU6fvvDVfRG,0.803,0.597,11,-6.035,0,0.1410,0.000619,0.000005,0.3230,0.331,136.008,audio_features,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4,Way 2 Sexy
4,6PQ88X9TkUIAUIZJHW2upE,0.808,0.897,11,-3.712,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,audio_features,6PQ88X9TkUIAUIZJHW2upE,spotify:track:6PQ88X9TkUIAUIZJHW2upE,https://api.spotify.com/v1/tracks/6PQ88X9TkUIA...,https://api.spotify.com/v1/audio-analysis/6PQ8...,231041,4,Bad Habits
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6G2xXQRSZlEzYsUEb3uvuR,0.405,0.565,1,-9.266,0,0.0372,0.380000,0.019500,0.1070,0.166,159.892,audio_features,6G2xXQRSZlEzYsUEb3uvuR,spotify:track:6G2xXQRSZlEzYsUEb3uvuR,https://api.spotify.com/v1/tracks/6G2xXQRSZlEz...,https://api.spotify.com/v1/audio-analysis/6G2x...,214149,3,Esta Danada
96,1tf4Bfk8JIKryLpWYRvyiI,0.653,0.623,8,-4.560,1,0.0244,0.508000,0.000003,0.2850,0.718,100.021,audio_features,1tf4Bfk8JIKryLpWYRvyiI,spotify:track:1tf4Bfk8JIKryLpWYRvyiI,https://api.spotify.com/v1/tracks/1tf4Bfk8JIKr...,https://api.spotify.com/v1/audio-analysis/1tf4...,206507,4,Whiskey And Rain
97,4s7QLoImIwmPi9L6dq1nVW,0.568,0.560,9,-9.059,1,0.4200,0.228000,0.000003,0.3900,0.246,130.019,audio_features,4s7QLoImIwmPi9L6dq1nVW,spotify:track:4s7QLoImIwmPi9L6dq1nVW,https://api.spotify.com/v1/tracks/4s7QLoImIwmP...,https://api.spotify.com/v1/audio-analysis/4s7Q...,308756,4,TSU
98,4VCbgIdr8ptegWeJpqLVHH,0.730,0.576,2,-5.442,1,0.2870,0.354000,0.000036,0.1500,0.155,92.131,audio_features,4VCbgIdr8ptegWeJpqLVHH,spotify:track:4VCbgIdr8ptegWeJpqLVHH,https://api.spotify.com/v1/tracks/4VCbgIdr8pte...,https://api.spotify.com/v1/audio-analysis/4VCb...,228461,4,Love All


In [8]:
def clustering(df,n):
    """
    Explanations: with the dataframe (df) you assign, the fuction will use the standard
    scaler to scale the data set.
    And with the n value you provide, the fuction will do the clustering based on the n number of clusters you want.
    
    df: dataframe
    n: integer
    """
    df_cat = df[['track_id', 'songs']]
    df_num = df._get_numeric_data()
    
    # scale the numerical data
    X_prep = StandardScaler().fit_transform(df_num)
    X = pd.DataFrame(X_prep)
    
    # clustering
    kmeans = KMeans(n_clusters=n, random_state=42)
    kmeans.fit(X)
    
    cluster = kmeans.predict(X)
    clusters = pd.DataFrame(cluster)
    clusters.columns = ['cluster']
    
    # concat df_cat, X, cluster
    billboard_cluster = pd.concat([clusters, X, df_cat], axis = 1)
    return billboard_cluster
    
    

In [9]:
billboard_100_clustered = clustering(billboard_100,7)
billboard_100_clustered

,cluster,0,1,2,3,4,5,6,7,8,9,10,11,12,track_id,songs
0,2,-0.525274,0.774817,-1.156317,0.320455,0.623610,-0.661802,-0.816729,-0.107335,-0.554408,-0.153652,1.459106,-1.391690,0.072954,5HCyWlXZPP0y6Gqq8TgA20,Stay
1,4,0.555230,0.347960,-0.585296,-0.620206,-1.603567,-0.556920,-0.901535,-0.107335,-0.957444,1.696990,0.838497,0.226674,0.072954,27NovPIUIRrOZoCHxABJwK,Industry Baby
2,5,-0.093072,0.746360,-1.156317,-0.169666,0.623610,-0.589497,-0.460071,-0.107335,1.076024,1.470109,-1.340222,-0.929495,0.072954,3ZozFqJJlDBNIm4xqxn2ZD,Fancy Like
3,4,1.054498,-0.413269,1.698787,0.051206,-1.603567,0.074752,-0.994261,-0.107239,1.121737,-0.807605,0.403014,1.278125,0.072954,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy
4,4,1.091757,1.721018,1.698787,1.186351,-1.603567,-0.769067,-0.776210,-0.106664,1.434110,0.349046,0.092227,0.665676,0.072954,6PQ88X9TkUIAUIZJHW2upE,Bad Habits
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,-1.911301,-0.640926,-1.156317,-1.527638,-1.603567,-0.749998,0.793177,0.309556,-0.523932,-1.541634,1.146638,0.276221,-2.358848,6G2xXQRSZlEzYsUEb3uvuR,Esta Danada
96,5,-0.063265,-0.228298,0.842256,0.771972,0.623610,-0.851701,1.396244,-0.107264,0.832221,0.914026,-0.717433,0.100030,0.072954,1tf4Bfk8JIKryLpWYRvyiI,Whiskey And Rain
97,0,-0.696665,-0.676498,1.127766,-1.426486,0.623610,2.291564,0.077035,-0.107270,1.632200,-1.185741,0.216548,2.457441,0.072954,4s7QLoImIwmPi9L6dq1nVW,TSU
98,0,0.510520,-0.562669,-0.870807,0.340978,0.623610,1.234804,0.670679,-0.106568,-0.196322,-1.590569,-0.963087,0.606192,0.072954,4VCbgIdr8ptegWeJpqLVHH,Love All


In [10]:
def song_recommender(client_id, client_secret):
    """
    Explanations: In this function, user can enter the user's interested song
    The funtion will
     1. check if that song is in the billboard 100
     2. If 'yes', it will recommend you another song which are similar from billboard 100
     3. If 'no', it will recommend you a new song that is matching your interested song from billboard 100
    """
    
    # takes in the song as an input 
    song = input("What song do you like? ")
    #cluster = []
    
    #billboard_100 = spotify_api("47749bd8cec54f04955b9db8069fc8c6", "621e6603f06d4acfb95010a79f92fc79", "https://www.billboard.com/charts/hot-100")
    
    if song in billboard_100_clustered['songs'].values:
        cluster_num = billboard_100_clustered.loc[billboard_100_clustered.songs == song, 'cluster'].values[0]
        #cluster.append(cluster_num)
        print("The cluster number is", cluster_num)
    
        # filter the column with cluster number
        filtered_billboard = billboard_100_clustered[billboard_100_clustered['cluster'] == cluster_num]
    
                                        
        # randomly select a song and return it
        print("If you like",song, "perhaps you also like", filtered_billboard.songs.sample(n=1).values[0])
    
    else:
        
        # get the features of the song from spotify using the API
        sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

        results = sp.search(q='track: {}'.format(song), limit=2)
        
        # Get track_id
        tracks_id = (results['tracks']['items'][0]['id'])
        
        #fetch the audio features
        features = sp.audio_features(tracks = tracks_id)
        features_df = pd.DataFrame(features)
        
        # add the song(row) to the billboard_100 (this is NOT clustered)
        billboard_100_added = pd.concat([billboard_100, features_df], axis = 0)
        
        # scale (standard_scaler)
        # first separate the numerical and categorical
        billboard_100_cat = billboard_100_added[['track_id', 'songs']]
        billboard_100_cat = billboard_100_cat.reset_index().drop('index', axis =1)
        
        billboard_100_num = billboard_100_added._get_numeric_data()
        
        X_prep = StandardScaler().fit_transform(billboard_100_num)
        X_prep = pd.DataFrame(X_prep)
        
        
        # kmeans
        kmeans = KMeans(n_clusters= 11, random_state=42)
        kmeans.fit(X_prep)
        
        
        #clustering
        cluster = kmeans.predict(X_prep)
        clusters = pd.DataFrame(cluster)
        clusters.columns = ['cluster']
        
        #print(billboard_100_cat)
        
        billboard_clustered = pd.concat([clusters, X_prep, billboard_100_cat], axis = 1)
        #return billboard_clustered
        
        
        # recommend a song which has the same cluster number to the user's interested song
       
        cluster_num = billboard_clustered.iloc[100]['cluster']
    
        # filter the column with cluster number
        filtered_billboard = billboard_clustered[billboard_clustered['cluster'] == cluster_num]
    
                                        
        # randomly select a song and return it
        print("If you like",song, "perhaps you also like", filtered_billboard.songs.sample(n=1).values[0])
        
        
        
        

In [15]:
song_recommender("47749bd8cec54f04955b9db8069fc8c6", "621e6603f06d4acfb95010a79f92fc79")

What song do you like? mockingbird
If you like mockingbird perhaps you also like TSU
